# 교차검증 - K-Fold, Stratified K-Fold, cross_val_score()(API), GridSearchCV(하이퍼파라미터 튜닝)

## 과적합을 피하기 위한 K-Fold 교차검증, 
## 전체 레이블 값의 분포도를 반영하는StratifiedKFOLD
### KFOLD 데이터 학습 및 예측 진행순서 1) 폴드 세트 설정 2) for루프에서 반복 학습 및 테스트 데이터 인덱스 추출 3) 학습 및 예측 수행 후 성능 반환

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold # 이 부분 중요. 사용할 모델을 자체적으로 제공한다.
from sklearn.datasets import load_iris
import numpy as np

iris = load_iris()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state=156)

# 5개의 폴드 세트로 분리하는 KFold 객체와 폴드 세트별 정확도를 담을 리스트 객체 생성.
kfold = KFold(n_splits=5) # 5개의 폴드 객체 생성. 이후 split으로 전체 붓꽃 데이터를 5개의 폴드 데이터 세트로 분리.
cv_accuracy = [] # 정확도 담을 리스트 객체
print('붓꽃 데이터 세트 크기:', features.shape[0]) # shape는 크기를 나타냄

붓꽃 데이터 세트 크기: 150


In [8]:
# 학습용 데이터 세트는 120개, 검증 데이터 세트는 30개로 분할됨.
# KFold는 split()을 호출하면 학습용/검증용 데이터로 분할 할 수 있는 인덱스를 반환한다. 이를 바탕으로 데이터 추출은 반환된 인덱스를 기반으로 개발 코드에서 직접 수행해야 한다.

n_iter = 0   # 반복을 위해 선언한 변수. 아래에서 +1 씩 증감된다.

# KFold 객체의 split()를 호출하면 FOLD별 학습용, 검증용 테스트의 ROW(행) 인덱스를 array로 반환
for train_index, test_index in kfold.split(features): #iris.data를 features 변수에 넣음
    # kfold.split()으로 반환된 인덱스를 이용해 학습용(test), 검증용(Vali) 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    # 학습 및 예측
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    n_iter += 1
    # 반복 시마다 정확도 측정
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    train_size = X_train.shape[0] # 유형
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'.format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter, test_index))
    cv_accuracy.append(accuracy)  # 정확도를 담을 리스트 객체를 수정해 나감
    
# 개별 iteration별 정확도를 합하여 평균 정확도 계산
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy))


# KFold에서는 레이블 데이터 값의 분포에 따라 학습 및 테스트 데이터 세트 설정이 잘 안되는 경우도 있다.


#1 교차 검증 정확도 :1.0, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#1 검증 세트 인덱스:[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차 검증 정확도 :0.9667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#2 검증 세트 인덱스:[30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차 검증 정확도 :0.8667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#3 검증 세트 인덱스:[60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차 검증 정확도 :0.9333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#4 검증 세트 인덱스:[ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 교차 검증 정확도 :0.7333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#5 검증 세트 인덱스:[120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

## 평균 검증 정확도: 0.9


# K-Fold 교차검증 문제점

In [13]:
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label']=iris.target
iris_df['label'].value_counts()


# 2    50   레이블 값은 세 품종 다 50으로 동일
# 1    50
# 0    50

2    50
1    50
0    50
Name: label, dtype: int64

In [19]:
kfold = KFold(n_splits=3)
n_iter =0
for train_index, test_index in kfold.split(iris_df):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())
    iris_df['label'].value_counts()
    
# 이 경우, 값이 50으로 동일하여 학습값과 검증값이 전혀 다른 것이 나온다. 값으로만 일치시키기 때문. (아래 참조)

## 교차 검증: 1
학습 레이블 데이터 분포:
 2    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    50
Name: label, dtype: int64
## 교차 검증: 2
학습 레이블 데이터 분포:
 2    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    50
Name: label, dtype: int64
## 교차 검증: 3
학습 레이블 데이터 분포:
 1    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    50
Name: label, dtype: int64


In [20]:
## 교차 검증: 1
#학습 레이블 데이터 분포:
# 2    50
#1    50
#Name: label, dtype: int64
#검증 레이블 데이터 분포:
# 0    50
#Name: label, dtype: int64
## 교차 검증: 2
#학습 레이블 데이터 분포:
# 2    50
#0    50
#Name: label, dtype: int64
#검증 레이블 데이터 분포:
# 1    50
#Name: label, dtype: int64
## 교차 검증: 3
#학습 레이블 데이터 분포:
# 1    50
#0    50
#Name: label, dtype: int64
#검증 레이블 데이터 분포:
# 2    50
#Name: label, dtype: int64

# 전체 레이블 값의 분포도를 반영. StratifiedKFOLD
## 원본 데이터의 레이블 분포도 특성을 반영한 학습 및 검증 데이터 세트를 만들 수 있으므로,
### 왜곡된 레이블 데이터 세트에서는 반드시 Stratified K Fold를 이용해 교차 검증해야 하다.

In [21]:
# StratifiedKFOLD는 레이블 데이터 분포도에 따라 학습/검증 데이터를 나누기 때문에,
# split() 메서드에 인자로 피처 데이터 세트뿐만 아니라 레이블 데이터 세트도 반드시 필요하다.


In [22]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)   # stratifiedKFold로 3개 객체 만들어서 분할.
n_iter=0

for train_index, test_index in skf.split(iris_df, iris_df['label']):   # split에 피쳐, 레이블 dataset 넣은 모습.
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())
    
# 예측이 아니라 실제를 보여주는 결과.

## 교차 검증: 1
학습 레이블 데이터 분포:
 2    33
1    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    17
1    17
0    17
Name: label, dtype: int64
## 교차 검증: 2
학습 레이블 데이터 분포:
 2    33
1    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    17
1    17
0    17
Name: label, dtype: int64
## 교차 검증: 3
학습 레이블 데이터 분포:
 2    34
1    34
0    34
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    16
1    16
0    16
Name: label, dtype: int64


In [28]:
# StratifiedKFold를 이용해 붓꽃 데이터 교차 검증

dt_clf=DecisionTreeClassifier(random_state=156)

skfold=StratifiedKFold(n_splits=3)
n_iter=0
cv_accuracy=[] # 단계별 정확도를 담을 리스트

# 다시한번 강조하지만, StratifiedKFold의 split() 호출 시 반드시 레이블 데이터 세트도 추가 입력 필요
for train_index, test_index in skfold.split(features, label):
    ############### split()으로 반환된 인덱스를 이용해 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]  # y가 label값
    
    # 학습 및 예측
    dt_clf.fit(X_train,y_train)
    pred = dt_clf.predict(X_test)
    
    # 반복 시마다 정확도 측정
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    train_size = X_train.shape[0]  # X_train 데이터가 담기는 양
    test_size = X_test.shape[0]    # X_test 데이터가 담기는 양
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'.format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter, test_index))
    cv_accuracy.append(accuracy)
    
# 교차 검증별 정확도 및 평균 정확도 계산
print('\n## 교차 검증별 정확도:',np.round(cv_accuracy, 4))   # 4는 소숫점 자리수
print('## 평균 검증 정확도:', np.mean(cv_accuracy))


#1 교차 검증 정확도 :0.9804, 학습 데이터 크기: 99, 검증 데이터 크기: 51
#1 검증 세트 인덱스:[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116]

#2 교차 검증 정확도 :0.9216, 학습 데이터 크기: 99, 검증 데이터 크기: 51
#2 검증 세트 인덱스:[ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133]

#3 교차 검증 정확도 :0.9792, 학습 데이터 크기: 102, 검증 데이터 크기: 48
#3 검증 세트 인덱스:[ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  84  85
  86  87  88  89  90  91  92  93  94  95  96  97  98  99 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

## 교차 검증별 정확도: [0.9804 0.9216 0.9792]
## 평균 검증 정확도: 0.9604


In [30]:
# cross_val_score() API. 정말 간단하게 구현 가능.
## Classification인 경우, Stratified K 폴드 방식으로 학습/테스트 세트 분할. 
## Regression인 경우, K-Fold 방식으로 분할

In [32]:
## Classification 방식이므로, Stratified K 폴드 방식으로 학습/테스트 세트 분할. 

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate  # API 호출 부분. 역시 model_selection 으로 호출한다.
from sklearn.datasets import load_iris

iris_data = load_iris()                                  # 데이터 셋 불러오고
dt_clf = DecisionTreeClassifier(random_state=156)       # 결정트리 사용한다고 하고

data = iris_data.data                                   # 데이터와 라벨 사용하고
label = iris_data.target

# 성능 지표는 정확도(accuracy), 교차 검증 세트는 3개
scores = cross_val_score(dt_clf, data, label, scoring='accuracy', cv=3)     # (알고리즘(estimator), X,Y, 성능지표(accuracy), 교차검증폴드수)   이것만 넣어주면 됨.
print('교차 검증별 정확도:', np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))


#비슷한 예로 cross_validate()가 있다. 여러 개의 평가 지표를 반환할 수 있으며 학습 데이터에 대한 성능 평가 지표와 수행 시간도 같이 제공한다.

교차 검증별 정확도: [0.9804 0.9216 0.9792]
평균 검증 정확도: 0.9604


In [33]:
# GridSearchCV API. 하이퍼 파라미터 튜닝 부분
 # -> 이는, 분류, 회귀 알고리즘에 사용되는 하이퍼 파라미터를 순차적으로 입력하면서 편리하게 최적의 파라미터를 도출할 수 있다.
 # -> 결정 트리 알고리즘의 하이퍼 파라미터를 순차적으로 변경하면서 최고 성능을 가지는 파라미터 조합도 찾을 수 있다.
 # -> 다음과 같이 순차적으로 바꿔 적용하며 테스트도 동시에 이뤄진다. (수행시간 오래걸림)
#순번 max_depth min_samples_split
#####1 1 2
#####2 1 3
#####3 2 2
#####4 2 3
#####5 3 2
#####6 3 3


# 3개 폴딩, 6개 파라미터 테스트 조합이면, cv 3회 * 파라미터 6개 조합 = 18회의 학습 /평가가 이뤄진다.

In [41]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# 데이터 로딩 및 학습 데이터와 테스트 데이터 분리
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, test_size=0.2, random_state=121)
# x, y,  학습데이터 80%, test데이터 20%, 학습데이터 값 고정 으로 설정)
dtree = DecisionTreeClassifier()

### 파라미터를 딕셔너리 형태로 설정
parameters = {'max_depth':[1,2,3], 'min_samples_split':[2, 3]}  # 총 6개 파라미터 수행



NameError: name 'train_test_split' is not defined

In [44]:

import pandas as pd

grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True)
# dtree = 결정나무분류 알고리즘 사용. estimator 부분
# param_grid의 하이퍼 파라미터를 3개의 train, test set fold로 나누어 (3폴딩) 테스트 수행 설정.
## refit=True가 default임. True이면 가장 좋은 파라미터 설정으로 재학습시킴

# 붓꽃 학습 데이터로 param_grid의 하이퍼 파라미터를 순차적으로 학습/평가
grid_dtree.fit(X_train, y_train)

# GridSearchCV 결과를 추출해 DataFrame으로 변환
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 'split0_test_score','split1_test_score', 'split2_test_score']]


# GridSearchCV 객체의 fit()을 수행하면 최고 성능을 나타낸 하이퍼 파라미터의 값과 그 때의 평가 결과 값이 각각 best_params_, best_score_속성에 기록
 # -> 즉 이것을, cv_results_의 rank_test_score가 1일 때의 값이다
    
print('GridSearchCV 최적 파라미터:', grid_dtree.best_params_)
print('GridSearchCV 최고 정확도:{0:.4f}'.format(grid_dtree.best_score_))

# 또한, refit=True 이면, GridSearchCV가 최적 성능을 나타내는 하이퍼 파라미터로 Estimator를 학습해, best_estimator_로 저장한다.
 # -> 이를 이용해, train_test_split()으로 분리한 테스트 데이터 세트에 대해 예측하고 성능을 평가한다.

# GridSearchCV의 refit으로 이미 학습된 estimator 반환
estimator = grid_dtree.best_estimator_

# GridSearchCV의 best_estimator_는 이미 최적 학습이 됐으므로, 이걸 이용해 분리된 테스트 데이터 세트에 대해 예측하고 성능을 평가
pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))

NameError: name 'dtree' is not defined